# Business Filings in NYC, Brooklyn, and Red Hook

In this notebook, I analyzed business activity in Red Hook compared to other parts of NYC using data from the "Legally Operating Businesses" dataset posted on NYC Open data:

https://data.cityofnewyork.us/Business/Legally-Operating-Businesses/w7w3-xahh

It contains historical information on business filings in NYC including the business type and location.

## Libraries

In [1]:
import urllib.request
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon
from fiona.crs import from_epsg
import matplotlib.pylab as plt
import numpy as np
import copy

## Read in the data and clean it up

In [2]:
# Read in the data
business_filings = pd.read_csv('data/primary_source_data/business_filings.csv')
business_filings.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DCA License Number,License Type,License Expiration Date,License Status,License Creation Date,Industry,Business Name,Business Name 2,Address Building,Address Street Name,...,Community Board,Council District,BIN,BBL,NTA,Census Tract,Detail,Longitude,Latitude,Location
0,0959361-DCA,Business,03/31/2021,Active,04/25/1997,Garage,"PARKANLEX GARAGE COMPANY, LLC",NaN,115,E 75TH ST,...,108.0,4.0,1.04314e+06,1014100011,MN40,128.0,"Vehicle Spaces: 165, Bicycle Spaces: 17",-73.961625,40.772737,"(40.7727369020104, -73.96162515178469)"
1,1235988-DCA,Business,02/28/2021,Active,08/15/2006,Home Improvement Contractor,"JUNIOR'S IRONWORK & CONSTRUCTION, INC.",NaN,16927,LIBERTY AVE,...,412.0,27.0,4e+06,4102180032,QN61,444.0,NaN,-73.788915,40.703297,"(40.70329689832498, -73.78891454010119)"
2,1453298-DCA,Individual,05/31/2019,Active,01/07/2013,Locksmith,"RAO, GARY PETER",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2049399-DCA,Business,12/31/2020,Active,03/09/2017,Electronics Store,"CANAL ALARM DEVICES, INC.",NaN,387,CANAL ST,...,102.0,1.0,1.00294e+06,1002270004,MN24,47.0,NaN,-74.004791,40.721466,"(40.721465796925365, -74.00479091086221)"
4,2021443-DCA,Business,03/31/2021,Active,04/21/2015,Garage and Parking Lot,O & A PARKING GARAGE INC.,NaN,1100,FRANKLIN AVE,...,NaN,NaN,NaN,NaN,BX35,185.0,"Vehicle Spaces: 36, Bicycle Spaces: 10",NaN,NaN,NaN


In [3]:
# Remove rows for filings with Address Borough listed as "Outside NYC"
nyc_filings = business_filings[business_filings['Address Borough']!='Outside NYC']

In [4]:
# Drop "Location" column since it is improperly formatted
nyc_filings.drop(['Location'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
# Convert creation date to datetime
nyc_filings['License Creation Date'] = pd.to_datetime(nyc_filings['License Creation Date'], \
                                                           format='%m/%d/%Y')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
print("Total number of business filings in all of NYC: {}".format(nyc_filings.shape[0]))

Total number of business filings in all of NYC: 184145


In [7]:
print("Number missing lon lat info: {}".format(nyc_filings.Longitude.isnull().sum()))

Number missing lon lat info: 70523


## Grouping by date for NYC

In [8]:
# All filings in NYC
filings_date_groups = nyc_filings.set_index('License Creation Date')
filings_date_groups.head()

,DCA License Number,License Type,License Expiration Date,License Status,Industry,Business Name,Business Name 2,Address Building,Address Street Name,Secondary Address Street Name,...,Borough Code,Community Board,Council District,BIN,BBL,NTA,Census Tract,Detail,Longitude,Latitude
License Creation Date,,,,,,,,,,,,,,,,,,,,,
1997-04-25,0959361-DCA,Business,03/31/2021,Active,Garage,"PARKANLEX GARAGE COMPANY, LLC",NaN,115,E 75TH ST,NaN,...,1.0,108.0,4.0,1.04314e+06,1014100011,MN40,128.0,"Vehicle Spaces: 165, Bicycle Spaces: 17",-73.961625,40.772737
2006-08-15,1235988-DCA,Business,02/28/2021,Active,Home Improvement Contractor,"JUNIOR'S IRONWORK & CONSTRUCTION, INC.",NaN,16927,LIBERTY AVE,NaN,...,4.0,412.0,27.0,4e+06,4102180032,QN61,444.0,NaN,-73.788915,40.703297
2013-01-07,1453298-DCA,Individual,05/31/2019,Active,Locksmith,"RAO, GARY PETER",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-09,2049399-DCA,Business,12/31/2020,Active,Electronics Store,"CANAL ALARM DEVICES, INC.",NaN,387,CANAL ST,NaN,...,1.0,102.0,1.0,1.00294e+06,1002270004,MN24,47.0,NaN,-74.004791,40.721466
2015-04-21,2021443-DCA,Business,03/31/2021,Active,Garage and Parking Lot,O & A PARKING GARAGE INC.,NaN,1100,FRANKLIN AVE,NaN,...,2.0,NaN,NaN,NaN,NaN,BX35,185.0,"Vehicle Spaces: 36, Bicycle Spaces: 10",NaN,NaN


In [9]:
# Grouping all filings by the month of the creation date and taking the count of each column
filings_count = filings_date_groups.resample('M').mean()
filings_count.reset_index(inplace=True)
filings_count['Year'] = filings_count['License Creation Date'].dt.year
filings_count['Year'].astype(int, inplace=True)
filings_count.head()

,License Creation Date,Borough Code,Community Board,Council District,Census Tract,Longitude,Latitude,Year
0,1977-01-31,3.0,301.0,33.0,561.0,-73.952437,40.726685,1977
1,1977-02-28,NaN,NaN,NaN,NaN,NaN,NaN,1977
2,1977-03-31,NaN,NaN,NaN,NaN,NaN,NaN,1977
3,1977-04-30,4.0,403.0,21.0,357.0,-73.868315,40.768801,1977
4,1977-05-31,NaN,NaN,NaN,NaN,NaN,NaN,1977


## For normalization by population

In [10]:
# Set a multiplier for the normalized fields since they will otherwise be very small
norm_multiplier_pop = 1

In [11]:
# Population data over time
pop_df = pd.read_csv('data/created_data/pop_time.csv')
pop_df

,Year,New York County,Kings County,Bronx County,Queens County,Richmond County,New York City Total,Red Hook CTs
0,2010,1583345,2466782,1365725,2199169,463450,8078471,10973
1,2011,1588257,2486119,1374593,2213977,466034,8128980,11374
2,2012,1596735,2512740,1386364,2235008,468374,8199221,10987
3,2013,1605272,2539789,1397315,2256400,470223,8268999,10670
4,2014,1618398,2570801,1413566,2280602,471522,8354889,10440
5,2015,1629507,2595259,1428357,2301139,472481,8426743,10768
6,2016,1634989,2606852,1436785,2310011,473324,8461961,11102
7,2017,1653877,2635121,1455846,2339280,475948,8560072,11316


In [12]:
# Merge the two
file_count_merge = filings_count.merge(pop_df, how='left', left_on='Year', right_on='Year')
file_count_merge.head()

,License Creation Date,Borough Code,Community Board,Council District,Census Tract,Longitude,Latitude,Year,New York County,Kings County,Bronx County,Queens County,Richmond County,New York City Total,Red Hook CTs
0,1977-01-31,3.0,301.0,33.0,561.0,-73.952437,40.726685,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1977-02-28,NaN,NaN,NaN,NaN,NaN,NaN,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1977-03-31,NaN,NaN,NaN,NaN,NaN,NaN,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1977-04-30,4.0,403.0,21.0,357.0,-73.868315,40.768801,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1977-05-31,NaN,NaN,NaN,NaN,NaN,NaN,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Get the normalized count
file_count_merge['count_norm_pop'] = file_count_merge['DCA License Number']/ \
    file_count_merge['New York City Total']*norm_multiplier_pop

KeyError: 'DCA License Number'

In [ ]:
# Plot
plt.figure(figsize=(20,8))
plt.plot(file_count_merge['License Creation Date'], file_count_merge['count_norm_pop'])
plt.title("Count of Businesses on File Each Month in NYC", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count per Month (for every {} resident(s))".format(norm_multiplier_pop), fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

## Business Filings for Brooklyn only

In [ ]:
# Create a Brooklyn df
bk_df = nyc_filings[(nyc_filings['Address Borough']=='Brooklyn') | (nyc_filings['Address Borough']=='BROOKLYN')]

In [ ]:
# Repeat the steps above
bk_date_groups = bk_df.set_index('License Creation Date')
bk_count = bk_date_groups.resample('M').mean()
bk_count.reset_index(inplace=True)
bk_count['Year'] = bk_count['License Creation Date'].dt.year
bk_merge = pd.merge(bk_count, pop_df[['Kings County', 'Red Hook CTs', 'Year']], how='left', \
                    left_on='Year', right_on='Year')
bk_merge['count_norm_pop'] = bk_merge['DCA License Number']/bk_merge['Kings County']*norm_multiplier_pop
bk_merge.head()

In [ ]:
# Plot
plt.figure(figsize=(20,8))
plt.plot(bk_merge['License Creation Date'], bk_merge['count_norm_pop'])
plt.title("Count of Monthly Business Filings in Brooklyn", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count per Month (for every {} resident(s))".format(norm_multiplier_pop), fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

## Brooklyn and all of NYC

In [ ]:
# Brooklyn and all of NYC
plt.figure(figsize=(20,8))
plt.plot(bk_merge['License Creation Date'], bk_merge['count_norm_pop'], label='Brooklyn')
plt.plot(file_count_merge['License Creation Date'], file_count_merge['count_norm_pop'], label='NYC')
plt.title("Count of Businesses on File Each Month and NYC", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count per Month (for every {} resident(s))".format(norm_multiplier_pop), fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

## Business Filings for Red Hook only

I read in the shapefile created in the "geoscope" notebook in order to locate businesses located specifically in Red Hook based on their lon lat locations. Then I compare them to Brooklyn and NYC.

In [ ]:
# Read in the Red Hook shapefile
rh_tracts = gpd.read_file('data/created_data/rh_comb_tract/rh_comb_tract.shp')
rh_tracts

In [ ]:
# Make a copy of business filings for Red Hook only
rh_df = copy.deepcopy(nyc_filings)

In [ ]:
# Remove rows with null lat lon data
rh_df = rh_df[rh_df['Longitude'].notnull()]
rh_df['lonlat'] = list(zip(rh_df.Longitude, rh_df.Latitude))

In [ ]:
# Make a geodataframe
rh_df = gpd.GeoDataFrame(rh_df)
rh_df['geometry'] = rh_df[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))
rh_df.head()

In [ ]:
# Match CRS
rh_df.crs = from_epsg(4326)
rh_df = rh_df.to_crs(epsg=2263)

In [ ]:
# Join with the Red Hook census tract to keep only business filings within the neighborhood
rh_df_sjoin = gpd.tools.sjoin(rh_df,rh_tracts)

In [ ]:
# Bring in more data for plotting
rh_centroid = gpd.read_file('data/created_data/rh_centroid/rh_centroid.shp')
site_poly = gpd.read_file('data/created_data/site/site_poly/site_poly.shp')
site_centroid = gpd.read_file('data/created_data/site/site_centroid/site_centroid.shp')

In [ ]:
# Plot
f, ax = plt.subplots(figsize=(8,8))
rh_tracts.plot(alpha=1,linewidth=0.8,ax=ax,color='w',edgecolor='black')
rh_df_sjoin.plot(alpha=.5, linewidth=0.2,ax=ax,color='r',edgecolor='black',markersize=125, label='Businesses')
rh_centroid.plot(alpha=.5, linewidth=0.2,ax=ax,color='g',edgecolor='black',markersize=125,label='RH Centroid')
site_poly.plot(alpha=.2, linewidth=0.2,ax=ax,color='b',edgecolor='black',markersize=100)
site_centroid.plot(alpha=.5, linewidth=0.2,ax=ax,color='b',edgecolor='black',markersize=125, \
                   label='Project Site')
plt.title("Business Filings Within Red Hook", fontsize=20)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.legend(fontsize=12, loc='upper left')
plt.show()

### Note:

The plot of locations above is less useful because many businesses are missing longitude latitude information. I would recommend using a Google Geocode search (see the "zap" notebook for a method for geocode searching) to try and pick up more of this location information.

In [ ]:
# Number of business filings in Red Hook
rh_df_sjoin.shape[0]

In [ ]:
# Red Hook groups
rh_date_groups = rh_df_sjoin.set_index('License Creation Date')
rh_date_groups.head()

In [ ]:
# Grouping all filings by the month of the creation date and taking the count of each column
rh_count = rh_date_groups.resample('M').mean()
rh_count.reset_index(inplace=True)
rh_count['Year'] = rh_count['License Creation Date'].dt.year
rh_merge = pd.merge(rh_count, pop_df[['Red Hook CTs', 'Year']], how='left', \
                    left_on='Year', right_on='Year')
rh_merge['count_norm_pop'] = rh_merge['DCA License Number']/rh_merge['Red Hook CTs']*norm_multiplier_pop
rh_merge.head()

In [ ]:
# Brooklyn and all of NYC
plt.figure(figsize=(20,8))
plt.plot(bk_merge['License Creation Date'], bk_merge['count_norm_pop'], label='Brooklyn')
plt.plot(rh_merge['License Creation Date'], rh_merge['count_norm_pop'], c='r', label='Red Hook')
plt.plot(file_count_merge['License Creation Date'], file_count_merge['count_norm_pop'], c='purple', label='NYC')
plt.title("Count of Businesses on File Each Month", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count per Month (for every {} resident(s))".format(norm_multiplier_pop), fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=20)
plt.show()

## Business filings by industry

In [ ]:
# All unique industry names for filings across NYC
industries = set(rh_df['Industry'].values)
industries

In [ ]:
# All unique industry names for filings in Red Hook
rh_industries = set(rh_df['Industry'].values)
rh_industries

In [ ]:
# Function for making a dictionary of individual dfs with counts for the total filings of each industry
def industry_group_maker(industry_list):
    industry_dfs = {}
    for industry in industry_list:
        temp_df = rh_df[rh_df['Industry']==industry]
        temp_df = temp_df.set_index('License Creation Date')
        temp_df = temp_df.resample('M').mean()
        temp_df = temp_df[temp_df.index.year>=2010]
        industry_dfs[industry] = industry_dfs.get(industry, temp_df)
    return industry_dfs

In [ ]:
# Get all the individualized industry dataframes for Red Hook
all_rh_industry_dfs = industry_group_maker(rh_industries)

In [ ]:
all_rh_industry_dfs['Tobacco Retail Dealer']

In [ ]:
# Plot
f, ax = plt.subplots(figsize=(20,8))
for k,v in all_rh_industry_dfs.items():
    v['DCA License Number'].plot(alpha=1,linewidth=2,ax=ax,label=k)
plt.title("Businesses on File By Industry in Red Hook", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Number of Businss Filings per Month", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),ncol=5,fontsize=15)
plt.show()

In [ ]:
# Getting the top industries (for all time) since the previous plot is confusing
top_industries = {}
for k,v in all_rh_industry_dfs.items():
    top_industries[k] = top_industries.get(k, (v['DCA License Number'].sum(), v))
top_industries = sorted(top_industries.items(), key=lambda x: x[1][0], reverse=True)

In [ ]:
# Set the number of top industries you want
cutoff = 5

In [ ]:
print("Top {} industries and total number of filings since 2010:".format(cutoff))
print("")
for i in range(cutoff):
    print(str(top_industries[i][0]) + " - " + str(top_industries[i][1][0]))
    print("")

In [ ]:
# Plot the top X industries (X = cutoff number)
plt.figure(figsize=(20,8))
for i in range(cutoff):
    top_industries[i][1][1]['DCA License Number'].plot(alpha=1,linewidth=2, \
                                                       label=top_industries[i][0])
plt.title("Top {} Industries in Red Hook and Total Number of Filings".format(cutoff), fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Businesses on File Each Month", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc="upper left", fontsize=15)
plt.show()

### Note:

The above plot shows the top industries in Red Hook between 2010 and 2019. The top industries were found by taking those with the overall largest count of business filings between 2010 and 2019.

In [ ]:
# Removing laundries since it may be an outlier
top_ind_edit = {}
for k,v in all_rh_industry_dfs.items():
    if k != 'Laundries':
        top_ind_edit[k] = top_ind_edit.get(k, (v['DCA License Number'].sum(), v))
top_ind_edit = sorted(top_ind_edit.items(), key=lambda x: x[1][0], reverse=True)

In [ ]:
print("Top {} industries and total number of filings since 2010 (not including laundries:".format(cutoff))
print("")
for i in range(cutoff):
    print(str(top_ind_edit[i][0]) + " - " + str(top_ind_edit[i][1][0]))
    print("")

In [ ]:
# Plot top X industries without laundries
plt.figure(figsize=(20,8))
for i in range(cutoff):
    top_ind_edit[i][1][1]['DCA License Number'].plot(alpha=1,linewidth=2, \
                                                       label=top_ind_edit[i][0])
plt.title("Count of Top {} Industries on File in Red Hook".format(cutoff), fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count of Businesses on File", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc="upper left", fontsize=15)
plt.show()

## Normalization by area

In [ ]:
# Can use this to make the filings count normalized by shape area larger
norm_mult_area = 1

In [ ]:
# Pull in the census tracts to get shape areas
nyc_cts = gpd.read_file('data/primary_source_data/census_tracts/geo_export_a1169309-1083-403c-8f43-' + \
                        '77f62e2a399e.shp')
nyc_cts = nyc_cts.to_crs(epsg=2263)
nyc_cts.head()

In [ ]:
# Get total area of NYC
nyc_tot_area = nyc_cts.shape_area.sum()
nyc_tot_area

In [ ]:
# Get total area of Brooklyn
bk_tot_area = nyc_cts[nyc_cts['boro_name']=='Brooklyn'].shape_area.sum()
rh_tot_area = rh_tracts.shape_area.values[0]

In [ ]:
# Normalize the filings for all of NYC
file_count_merge['count_norm_area'] = file_count_merge['DCA License Number']/nyc_tot_area*norm_mult_area
file_count_merge.head()

In [ ]:
# Normalize the filings for Brooklyn and Red Hook
bk_merge['count_norm_area'] = bk_merge['DCA License Number']/bk_tot_area*norm_mult_area
rh_merge['count_norm_area'] = rh_merge['DCA License Number']/rh_tot_area*norm_mult_area

In [ ]:
# Brooklyn and all of NYC
plt.figure(figsize=(20,8))
plt.plot(bk_merge['License Creation Date'], bk_merge['count_norm_area'], label='Brooklyn')
plt.plot(rh_merge['License Creation Date'], rh_merge['count_norm_area'].rolling(6).mean(), c='r', label='Red Hook')
plt.plot(file_count_merge['License Creation Date'], file_count_merge['count_norm_area'], c='purple', label='NYC')
plt.title("Count of Businesses on File", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count per Month (normalized by shape area)".format(norm_multiplier_pop), fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=20)
plt.show()

In [ ]:
# To modify the rolling mean window for Red Hook
rh_roller = 6
nyc_bk_roller = 1

In [ ]:
# Brooklyn and all of NYC
plt.figure(figsize=(12,7))
plt.plot(file_count_merge[file_count_merge['Year']>=2010]['License Creation Date'], \
         file_count_merge[file_count_merge['Year']>=2010]['count_norm_area'].rolling(nyc_bk_roller).mean(), \
         c='g', label='NYC')
plt.plot(bk_merge[bk_merge['Year']>=2010]['License Creation Date'], \
         bk_merge[bk_merge['Year']>=2010]['count_norm_area'].rolling(nyc_bk_roller).mean(), c='b', label='Brooklyn')
plt.plot(rh_merge[rh_merge['Year']>=2010]['License Creation Date'], \
         rh_merge[rh_merge['Year']>=2010]['count_norm_area'].rolling(rh_roller).mean(), c='r', label='Red Hook')
plt.title("Count of Businesses on File Over Time", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Count of Businesses on File (normalized by land area)".format(norm_multiplier_pop), fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=20)
plt.show()

## Above plot

This is the monthly count of business filings in different NYC geographies. Right now, the plot shows a rolling mean for Red Hook only with a window of 6 and Brooklyn & NYC with a window of 1. We could change this setting depending on what makes the most sense.

# Next steps

* Narrow down the business filings
* Deal with missing latitude and longitude